# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuta,-8.7233,115.1723,25.97,100,20,3.60,ID,1717942079
1,1,adamstown,-25.0660,-130.1015,22.09,72,6,6.16,PN,1717942079
2,2,okhotsk,59.3833,143.3000,3.72,92,0,2.22,RU,1717942080
3,3,mooloolaba,-26.6833,153.1167,12.95,95,0,2.81,AU,1717942081
4,4,east london,-33.0153,27.9116,18.52,55,0,1.54,ZA,1717941940


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng','Lat',
    geo = True,
    tiles = 'OSM',
    color='City',
    size = 'Humidity',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    alpha = 0.7,
    hover_cols = ['City','Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
Criteria_cities_df = city_data_df[(city_data_df['Max Temp'] < 30) &
                                  (city_data_df['Humidity'] >= 40) &
                                  (city_data_df['Humidity'] <= 60) &
                                  (city_data_df['Cloudiness'] >= 40) &
                                  (city_data_df['Cloudiness'] <= 60) &
                                  (city_data_df['Wind Speed'] < 9)]

# Drop any rows with null values
Criteria_cities_df = Criteria_cities_df.dropna()


# Display sample data
Criteria_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
126,126,kalemie,-5.9475,29.1947,26.79,44,57,5.04,CD,1717942198
251,251,kurmuk,10.5500,34.2833,29.82,56,55,3.77,SD,1717942302
333,333,klyuchi,52.2667,79.1667,21.13,57,52,5.50,RU,1717942365
355,355,mezhdurechenskiy,59.6000,65.9333,20.70,53,55,2.18,RU,1717942384
467,467,crane,31.3974,-102.3501,26.57,56,41,8.73,US,1717942329
481,481,dergachi,51.2328,48.7659,25.72,49,45,5.39,RU,1717942497
540,540,astaneh-ye ashrafiyeh,37.2596,49.9444,25.96,52,49,1.49,IR,1717942540
553,553,mao,39.8885,4.2658,26.31,53,40,6.17,ES,1717942549
565,565,concepcion del oro,24.6333,-101.4167,18.25,44,40,1.25,MX,1717942557
566,566,sodankyla,67.4167,26.6000,18.10,40,52,2.58,FI,1717942557


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(Criteria_cities_df)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
126,126,kalemie,-5.9475,29.1947,26.79,44,57,5.04,CD,1717942198,
251,251,kurmuk,10.5500,34.2833,29.82,56,55,3.77,SD,1717942302,
333,333,klyuchi,52.2667,79.1667,21.13,57,52,5.50,RU,1717942365,
355,355,mezhdurechenskiy,59.6000,65.9333,20.70,53,55,2.18,RU,1717942384,
467,467,crane,31.3974,-102.3501,26.57,56,41,8.73,US,1717942329,
481,481,dergachi,51.2328,48.7659,25.72,49,45,5.39,RU,1717942497,
540,540,astaneh-ye ashrafiyeh,37.2596,49.9444,25.96,52,49,1.49,IR,1717942540,
553,553,mao,39.8885,4.2658,26.31,53,40,6.17,ES,1717942549,
565,565,concepcion del oro,24.6333,-101.4167,18.25,44,40,1.25,MX,1717942557,
566,566,sodankyla,67.4167,26.6000,18.10,40,52,2.58,FI,1717942557,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accommodation.hotel",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}",
    params["bias"] =  f"proximity:{lng},{lat}"   

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"



    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kalemie - nearest hotel: Hotel du Lac
kurmuk - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
mezhdurechenskiy - nearest hotel: КедрОтель
crane - nearest hotel: No hotel found
dergachi - nearest hotel: No hotel found
astaneh-ye ashrafiyeh - nearest hotel: هتل دهدار
mao - nearest hotel: Hevresac
concepcion del oro - nearest hotel: No hotel found
sodankyla - nearest hotel: Hotelli Karhu


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
126,126,kalemie,-5.9475,29.1947,26.79,44,57,5.04,CD,1717942198,Hotel du Lac
251,251,kurmuk,10.5500,34.2833,29.82,56,55,3.77,SD,1717942302,No hotel found
333,333,klyuchi,52.2667,79.1667,21.13,57,52,5.50,RU,1717942365,No hotel found
355,355,mezhdurechenskiy,59.6000,65.9333,20.70,53,55,2.18,RU,1717942384,КедрОтель
467,467,crane,31.3974,-102.3501,26.57,56,41,8.73,US,1717942329,No hotel found
481,481,dergachi,51.2328,48.7659,25.72,49,45,5.39,RU,1717942497,No hotel found
540,540,astaneh-ye ashrafiyeh,37.2596,49.9444,25.96,52,49,1.49,IR,1717942540,هتل دهدار
553,553,mao,39.8885,4.2658,26.31,53,40,6.17,ES,1717942549,Hevresac
565,565,concepcion del oro,24.6333,-101.4167,18.25,44,40,1.25,MX,1717942557,No hotel found
566,566,sodankyla,67.4167,26.6000,18.10,40,52,2.58,FI,1717942557,Hotelli Karhu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng','Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color='City',
    size = 'Humidity',
    alpha = 0.7,
    hover_cols = ['City','Humidity','Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)